In [1]:
import pandas as pd
import numpy as np
import random
import plotly.express as px
import plotly.graph_objects as go

# TODO add more distributions
# TODO estimate f(iteration) = len(_points)(iteration)
# TODO optimise algorithm
# TODO time check
# TODO exception: one point left, what to do

In [189]:
class Box(object):
  """Box with dots, the problem is to find their minimal convex shell"""
  def __init__(self, dimension=2, size=1, distribution='Uniform', points_number=100):
    self.dimension = dimension
    self.size = size
    self.distribution = distribution
    self.points_number = points_number
    self.convex_shell = []
    
    if distribution=='Uniform':
        points_array = np.array([
                      [random.random() for x_iterator in range(self.points_number)], 
                      [random.random() for y_iterator in range(self.points_number)]
                      ]).reshape((self.points_number,2))
    elif distribution=='Gauss':
        points_array = np.array([
                      [random.gauss(mu=1, sigma=1) for x_iterator in range(self.points_number)], 
                      [random.gauss(mu=1, sigma=1) for y_iterator in range(self.points_number)]
                      ]).reshape((self.points_number,2))
    self._points = pd.DataFrame(data=points_array, columns=("x", "y"))
    self.picture_data = px.scatter(self._points, x="x", y="y").data
                   

  def find_convex(self):
    def metric(vectors_array, vector):
        """Array of normalized dot products of each vec1 in vectors_array with single vector"""
        nvector = vector / np.linalg.norm(vector)
        nvectors_array = vectors_array / np.linalg.norm(vectors_array, axis=0)
        nvectors_array = np.column_stack((nvectors_array[0], nvectors_array[1]))
        return np.tensordot(nvectors_array, nvector, axes=1)
    
    data = self._points
    # The first point in the convex_shell is the lowest one
    lowest_point_index = (data.idxmin(axis=0).y)
    # print("_______________________________            lowest_point is ", lowest_point_index)
    self.convex_shell = [lowest_point_index]
    current_vector = np.array([1, 0])
    current_point_index = lowest_point_index
    looped_to_lowest = False
    # Then we go around points box counterclockwise: 
    while (looped_to_lowest == False):   
        difference_vectors = np.array([
            data.x - data.x[current_point_index],
            data.y - data.y[current_point_index]
        ])
        # Difference vector contains (current_point - current_point) = 0. Let's delete it:
        probe_vectors = np.delete(difference_vectors, (current_point_index), axis=1)
        metrics = metric(probe_vectors, current_vector)
        # Manually incert (current_point - current_point) metric to be the smallest and impossible: -2
        metrics = np.insert(metrics, current_point_index, -2)
        # print(len(metrics))
        next_point_index = np.where(metrics == np.amax(metrics))[0][0]
        # print(next_point_index)
        
        self.convex_shell.append(next_point_index)
        current_vector = np.array([
            data.x[next_point_index] - data.x[current_point_index],
            data.y[next_point_index] - data.y[current_point_index]
        ])
        current_point_index = next_point_index
        if current_point_index == lowest_point_index:
          looped_to_lowest = True
            
  def collect_convex(self, show=False):
    """Adds a new convex shell line to the figure data"""
    # TODO colormap for layers
    if len(self.convex_shell):
      convex_shell_df = self._points.loc[self._points.index[self.convex_shell]]
    else:
        convex_shell_df = pd.DataFrame(data=[], columns=("x", "y"))
        
    convex_shell_figure = px.line(convex_shell_df, x="x", y="y" )
    self.picture_data += convex_shell_figure.data
    if show:
        fig = go.Figure(data = self.picture_data)
        fig.show()
    
  def drop_convex(self):
    """Remove convex_shell from _points DataFrame"""
    self._points.drop(self.convex_shell, inplace=True, axis=0)
    self._points.reset_index(drop=True, inplace=True)
    self.convex_shell = []
      

In [168]:
box = Box(dimension=2, size=1, distribution='Gauss', points_number=4)
box.find_convex()
box.collect_convex()
box.drop_convex()


3
2
0
1


In [193]:
shell_list = []
for i in range(20):
    box = Box(dimension=2, size=1, distribution='Gauss', points_number=20)
    shell_size = []
    while box._points.shape[0]:
        box.find_convex()
        box.collect_convex(show=False)
        shell_size.append(len(box.convex_shell))
        box.drop_convex()
    # box.collect_convex(show=True)
    shell_list.append(shell_size)


In [194]:
df = pd.DataFrame(shell_list)
df = df.transpose()
df = pd.melt(df, ignore_index=False, var_name="Experiment", value_name="Size")
df['Iteration'] = df.index
df.reset_index(inplace=True)
df.dropna(inplace=True)
df.head()
chart = px.line(df, x='Iteration', y='Size', color='Experiment')
chart.show()